In [6]:
from fileToolbox import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [7]:
def warn(*args, **kwargs):
    pass


import warnings

warnings.warn = warn
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
set_of_person_ids = set(train_df["person_id"])
list_of_unique_person_ids = list(set_of_person_ids) # osoby to będą moje x

In [9]:
set_of_movie_ids = set(train_df["movie_id"])
list_of_unique_movie_ids = list(set_of_person_ids) # osoby to będą moje x

In [10]:
tmp_p = set(task_df["person_id"])
tmp_m = set(task_df["movie_id"])

Sprawdzenie czy napewno wszystkie person_id i movie_id z taska występują w trainie

In [11]:
for person in set_of_person_ids:
    if person not in tmp_p:
        print(person)

In [12]:
for movie in set_of_movie_ids:
    if movie not in tmp_m:
        print(movie)

Dla każdego filmu generuje n elementowy wektor cech

In [13]:
N = 10 # to będzie hiperparametr

In [14]:
M = len(set_of_movie_ids)
R = len(set_of_person_ids)

In [15]:
print(M)
print(R)

200
358


In [148]:
import random
# losowa inicjalizacja list p - parametrów i x - wejść z przedziału <-1, 1>
X_INPUTS = [[random.uniform(-1, 1) for j in range(N)] for i in range(M)] # parametrów jest n+1 bo stała
PARAMS = [[random.uniform(-1, 1) for j in range(N+1)] for i in range(R)]

In [19]:
# kodowanie filmów czyli m idzie od 0 do M, ale m to nie jest movie_id wieć muszę zrobić ,np. m=0 -> movie_id = 12
movies_dict = {}
for m in range(M):
    movies_dict[m] = list(set_of_movie_ids)[m]

In [ ]:
movies_dict

In [56]:
with open('json/movies_dict.json', 'w') as file:
    json.dump(movies_dict, file)

In [21]:
# kodowanie filmów czyli m idzie od 0 do M, ale m to nie jest movie_id wieć muszę zrobić ,np. m=0 -> movie_id = 12
person_dict = {}
for r in range(R):
    person_dict[r] = list(set_of_person_ids)[r]

In [ ]:
person_dict

In [55]:
with open('json/person_dict.json', 'w') as file:
    json.dump(person_dict, file)

In [50]:
def findMovieRatingByPerson(person_id, movie_id):
    person_movies = filterDataFrame("person_id", person_id, train_df)
    if movie_id in list(person_movies["movie_id"]):
        return int(person_movies[person_movies["movie_id"]==movie_id]["rate"])
    else:
        return None

In [59]:
# macierz Y - ustawić rate lub NULL na poszczególnych indeksach
Y_RESULTS = []
E = [] # tablica do przechowywania maski 0 i 1, czy m-ty film jest oceniony przez m-tą osobę
for m in range(M):
    movie_id = movies_dict[m]
    movie_ratings = []
    E_row = []
    for r in range(R):
        # znalezienie oceny dla m-tego filmu, wystawionej przez r-tą osobę
        person_id = person_dict[r]
        rate = findMovieRatingByPerson(person_id, movie_id)
        movie_ratings.append(rate)
        if rate is None:
            E_row.append(0)
        else:
            E_row.append(1)
    Y_RESULTS.append(movie_ratings)
    E.append(E_row)


In [29]:
def isMovieRatedByPerson(movie, person):
    df_person = filterDataFrame("person_id", person, train_df)
    list_of_movies = list(df_person["movie_id"]))
    if movie in list_of_movies:
        return 1
    else:
        return 0

In [2]:
# def main_function(p, x, y, alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
#     for epoch in range(num_of_epochs):
#         for r, p in enumerate(PARAMS):
#             for m in range(len(movies)): # m - liczba filmów, p - parametry da r-tej ososby
#                 for n in range(len(p)):
#                     p[n] = p[n] - alfa * derivative_P(x[m], y, p, n) # poprawiam naraz w jednej iteracji x i p
#                     x[m][n] = x[n] - beta * derivative_X(x[m], y, p, n)

In [146]:
def main_function(alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
    for epoch in range(num_of_epochs):
        print(Q(X_INPUTS, PARAMS, Y_RESULTS))
        for r, p in enumerate(PARAMS): # r - indeks osób
            for m, x in enumerate(X_INPUTS): # m - indeks filmów
                if E[m][r]==1: # używamy tylko tych danych, gdzie znany jest rate dla m-tego filmu i r-tej osoby
                    for n in range(len(p)):
                        # poprawiam naraz w jednej iteracji x i p
#                         print(str(n) + ' ' + str(derivative_P(X_INPUTS, Y_RESULTS, p, n, r)))
                        p[n] = p[n] - alfa * derivative_P(X_INPUTS, Y_RESULTS, p, n, r) # UWAGA parametrów jest n+1
                        if n != (len(p)-1): # p jest o 1 dłuższe niż x
                            x[n] = x[n] - beta * derivative_X(x, Y_RESULTS, PARAMS, n, m)
#         print(Q(X_INPUTS, PARAMS, Y_RESULTS)) # błąd liczymy po całej przestrzeni

In [121]:
def Q(X, P, Y):
    res = 0.0
    for r, p in enumerate(P): # r - indeks osób
            for m, x in enumerate(X): # m - indeks filmów
                if E[m][r]==1:
                    res += (calc_output(x, p) - Y[m][r]) ** 2
#                     print('calc: ' + str(calc_output(x, p)))
#                     print('y: ' + str(Y[m][r]))
#                     print("Res: " +str(res))
    return res / 2 

In [142]:
def derivative_P(X, Y, p, i, r):
    res = 0.0
    for m in range(len(X)): # przechodzimy po filmach m
        if E[m][r]==1:
            if i ==0: # dla p0 bez mnożenia z X[m][i]
                res += (calc_output(X[m], p) - Y[m][r])
            else:
                res += (calc_output(X[m], p) - Y[m][r]) * X[m][i-1]
    return (1 / (len(X))) * res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [141]:
def derivative_X(X, Y, p, i, m):
    res = 0.0
    for r in range(len(p)): # przechodzimy po osobach r
        if E[m][r]==1:
            res += (calc_output(X, p[r]) - Y[m][r]) * p[r][i+1]
    return (1 / (len(p))) * res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [125]:
def calc_output(x, p):
#     print(x)
#     print(p)
    summ = p[0]
    for n in range(len(x)):
        tmp = x[n]
        summ = summ + tmp * p[n+1] # muszę mnożyć x0 z p1, x1 z p2 itp. bo p o 1 dłuższe niż x
    return summ

In [149]:
main_function(0.9, 0.9, 100)

204974.64190780986
17540.704905987834
14169.101238115003
13632.990203194677
13409.557271787586
13284.357368043837
13207.247950317267
13159.62990553671
13130.52333247596
13112.024709598123
13099.292029702952
13089.718129589792
13081.9348182043
13075.199572478738
13069.092434289101
13063.374891666153
13057.923085836908
13052.69157386707
13047.685590918409
13042.934051299417
13038.46668310661
13034.301432180218
13030.442380590954
13026.882864795924
13023.60885833619
13020.601048958582
13017.836214737496
13015.288733463256
13012.93246767004
13010.742767250227
13008.698174784568
13006.781509057188
13004.980188431266
13003.285841814597
13001.69339744939
13000.199917839454
12998.803452360959
12997.502114131712
12996.293483536374
12995.174337842296
12994.140636586142
12993.187666230706
12992.310255824565
12991.503000644268
12990.760458071361
12990.077301161138
12989.448428550711
12988.86903592707
12988.334656565728
12987.8411784676
12987.384844648688
12986.962241909803
12986.570282224307
12986

In [150]:
with open('json/X_INPUTS_0.9_0.9_100.json', 'w') as file:
    json.dump(X_INPUTS, file)

In [151]:
with open('json/PARAMS_0.9_0.9_100.json', 'w') as file:
    json.dump(PARAMS, file)

In [168]:
# m- movie_id, r - person_id
def collaborativeFilteringClassifier(m, r):
    for key,value in movies_dict.items():
        if value==m:
            movie_id = key
    for key,value in person_dict.items():
        if value==r:
            person_id = key
    return round(calc_output(X_INPUTS[movie_id], PARAMS[person_id]))

In [152]:
train_list = []
for index, row in train_df.iterrows():
    train_list.append([int(row['person_id']), int(row['movie_id']), int(row['rate'])])

In [153]:
length = int(len(train_list)*0.2)

In [154]:
test = train_list[:length]

In [158]:
test_x = [elem[:2] for elem in test]
test_y = [elem[2] for elem in test]

In [169]:
results = []
for index, elem in enumerate(test_x):
    results.append(collaborativeFilteringClassifier(elem[1], elem[0]))

In [170]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, results)

In [171]:
round(accuracy, 2)

0.47

In [175]:
# wyniki są dla 100 epok, alfa,beta = 0.9, N=10
COUNTER = 0
def applyCollaborativeFilteringClassifier(row):
    global COUNTER
    print(COUNTER)
    COUNTER += 1
    movie_id = row['movie_id']
    person_id = row['person_id']    
    rate = collaborativeFilteringClassifier(movie_id, person_id)
    return rate

In [ ]:
# Apply the function to update the "rate" column
task_df['rate'] = task_df.apply(applyCollaborativeFilteringClassifier, axis=1)

In [177]:
task_df.to_csv('submission_collaborative_0.9_0.9_100.csv', index=False)

In [181]:
import random
N = 20
# losowa inicjalizacja list p - parametrów i x - wejść z przedziału <-1, 1>
X_INPUTS = [[random.uniform(-1, 1) for j in range(N)] for i in range(M)] # parametrów jest n+1 bo stała
PARAMS = [[random.uniform(-1, 1) for j in range(N+1)] for i in range(R)]

In [182]:
# 100 iteracji jak widać na poprzednim wywołaniu to za dużo po +/- 2- epoce już mało się zmienia
main_function(0.9, 0.9, 20)

222975.57277275823
14566.723969111647
9803.248514547884
8883.937559782433
8510.99210611923
8302.263369247039
8162.379320930968
8061.474574321414
7986.639768131271
7929.440080153594
7884.046683168882
7846.645359221159
7814.777585529416
7786.827432224494
7761.726023297432
7738.845963540526
7717.926229788669
7698.9285021558
7681.87012509295
7666.718296759361


In [183]:
results = []
for index, elem in enumerate(test_x):
    results.append(collaborativeFilteringClassifier(elem[1], elem[0]))

In [184]:
accuracy = accuracy_score(test_y, results)
print(round(accuracy, 2))

0.57


In [ ]:
task_df['rate'] = task_df.apply(applyCollaborativeFilteringClassifier, axis=1)

In [186]:
task_df.to_csv('submission_collaborative_a0.9_b0.9_e20_n20.csv', index=False)

In [187]:
with open('json/X_INPUTS_a0.9_b0.9_e20_n20.json', 'w') as file:
    json.dump(X_INPUTS, file)

In [188]:
with open('json/PARAMS_a0.9_b0.9_e20_n20.json', 'w') as file:
    json.dump(PARAMS, file)

In [189]:
N = 50
# losowa inicjalizacja list p - parametrów i x - wejść z przedziału <-1, 1>
X_INPUTS = [[random.uniform(-1, 1) for j in range(N)] for i in range(M)] # parametrów jest n+1 bo stała
PARAMS = [[random.uniform(-1, 1) for j in range(N+1)] for i in range(R)]

In [190]:
main_function(0.9, 0.9, 20)

273149.7611437246
9077.392038858314
3135.0162703698697
2151.291526446637
1731.1336422931925
1494.9933121792094
1341.7949146164665
1231.4047873518546
1145.87843759452
1076.5356060732006
1018.6677885138452
969.4405590642098
927.0509418105252
890.2780638446103
858.2025860157759
830.0726594944575
805.2634655200767
783.2639429661572
763.6590922881043
746.1064773971102


In [191]:
with open('json/X_INPUTS_a0.9_b0.9_e20_n50.json', 'w') as file:
    json.dump(X_INPUTS, file)

In [192]:
with open('json/PARAMS_a0.9_b0.9_e20_n50.json', 'w') as file:
    json.dump(PARAMS, file)

In [193]:
results = []
for index, elem in enumerate(test_x):
    results.append(collaborativeFilteringClassifier(elem[1], elem[0]))

In [194]:
accuracy = accuracy_score(test_y, results)
print(round(accuracy, 2))

0.97


In [ ]:
task_df['rate'] = task_df.apply(applyCollaborativeFilteringClassifier, axis=1)

In [196]:
task_df.to_csv('submission_collaborative_a0.9_b0.9_e20_n50.csv', index=False)